In [1]:
import networkx as nx
import random
import requests
import time
import math
from collections import deque, Counter

# -------------------------------------------------------------------------------------
# Hill Climb w/ random walk & Tabu Search + Simulated Annealing
# -------------------------------------------------------------------------------------
# 6.4.2025
# Podla zadania z uloh https://uuapp.plus4u.net/uu-managementkit-maing02/38744216cb324edca986789798259ba9/document?oid=67b7042a24292539ed1645df&pageOid=67b7043724292539ed1646c0
# Na spode pri main funkcii mozete najst aj color aj iscoloring funkciu
# Celkom dost som si pomahal AIckom, ale len ak som nedokazal spomenut syntax
# -------------------------------------------------------------------------------------
def load_web_graph(url):
    # nacitat graf z internetu, format DIMACS
    G = nx.Graph()
    response = requests.get(url)
    response.raise_for_status()

    for line in response.text.splitlines():
        if line.startswith("e"):
            _, u, v = line.split()
            G.add_edge(int(u), int(v))

    return G

def count_conflicts(G, col):
    # spocita konflikty
    conflicts = 0
    for u, v in G.edges():
        if col[u] == col[v]:
            conflicts += 1
    return conflicts

def get_conflicting_nodes(G, col):
    # vrati naspat nodes (po paroch) vedla seba s rovnakou farbou
    conflicting_nodes = set()
    for u, v in G.edges():
        if col[u] == col[v]:
            conflicting_nodes.add(u)
            conflicting_nodes.add(v)
    return conflicting_nodes

def random_walk(G, col, k, steps=1):
    # vykona random walk na konfliktnych nodoch (priorita)
    new_col = col.copy()
    
    for _ in range(steps):
        # priorita na konflitkne nodes
        conflicting_nodes = list(get_conflicting_nodes(G, new_col))
        
        if conflicting_nodes:
            # vyberie nahodny konfliktujuci node
            node = random.choice(conflicting_nodes)
        else:
            # ak nie su konflikty, ide na nahodny
            node = random.choice(list(G.nodes()))
            
        # ber farbu susedov
        neighbor_colors = {new_col[nbr] for nbr in G.neighbors(node)}
        
        # hladaj novu farbu, ktoru nepozuivaju susedia
        available_colors = list(set(range(k)) - neighbor_colors)
        
        if available_colors:
            # ak je to mozne, vyber farbu, ktora nekonfliktuje (s ostatnymi nodes)
            new_col[node] = random.choice(available_colors)
        else:
            # ak to neni mozne, vyber nahodnu farbu
            current_color = new_col[node]
            other_colors = [c for c in range(k) if c != current_color]
            if other_colors:
                new_col[node] = random.choice(other_colors)
    
    return new_col

def is_stagnating(history, threshold=10):
    # kontrola stagnacie algoritmu - ak sa 10x opakuje rovnaky vysledok
    if len(history) < 20:
        return False
    
    # spocitaj vyskyt konfliktov z historie
    counter = Counter(history)
    
    # ak sa hociaky konflikt opakuje viac ako threshold tak stagnuje
    return any(count >= threshold for count in counter.values())

def hill_climbing_with_walks(G, col, k, max_steps, phase_length=1000):
    # hill climb s random walk aby odislo z lokalneho minima
    # hill climb preto, lebo kontrolujeme ci to pomoze, a ak ano tak nechame, ak nie tak to ide do prec
    # ziadna inspiracia tu som skusal bordel vsetok co ma napadol
    n = len(G.nodes())
    
    best_col = col.copy()
    best_conflicts = count_conflicts(G, col)
    current_col = col.copy()
    current_conflicts = best_conflicts
    
    # nechaj historiu aby sme vedeli ci to stagnuje
    conflict_history = deque(maxlen=20)
    
    for step in range(max_steps):
        # ukaz progress aby sme vedeli co sa deje
        if step % 500 == 0:
            print(f"Step {step}: Best conflicts = {best_conflicts}, Current conflicts = {current_conflicts}")
        
        # ak nie su konflikty, tak hura pijeme pivo (osobne skor slivovica)
        if best_conflicts == 0:
            return best_col, True
        
        # kontrola stagnacie - ak tam je tak urob random walk
        if is_stagnating(conflict_history):
            if step % 500 == 0:
                print(f"Stagnation detected at step {step}. Taking a walk.")
            current_col = random_walk(G, best_col, k, steps=n//5)
            current_conflicts = count_conflicts(G, current_col)
            conflict_history.clear()
            continue
        
        # periodicke menenie medzi hill climb a random walk, lepsie to funguje
        if step % phase_length < phase_length * 0.8:  # 80% hill climb
            # priorita na conflicting nodes ako v random walk
            conflicting_nodes = list(get_conflicting_nodes(G, current_col))
            nodes_to_try = conflicting_nodes if conflicting_nodes else list(G.nodes())
            
            # skus nahodny node z conflicting nodes
            node = random.choice(nodes_to_try)
            current_color = current_col[node]
            
            # skus vsetky farby, okrem tej co je teraz
            best_node_color = current_color
            best_node_conflicts = current_conflicts
            
            for new_color in range(k):
                if new_color == current_color:
                    continue
                    
                current_col[node] = new_color
                conflicts = count_conflicts(G, current_col)
                
                if conflicts < best_node_conflicts:
                    best_node_color = new_color
                    best_node_conflicts = conflicts
            
            # ak najdeme lepsiu, nechaj
            current_col[node] = best_node_color
            current_conflicts = best_node_conflicts
            
        else:  # 20% random walk
            # rob mensie random walks
            current_col = random_walk(G, current_col, k, steps=max(1, n//20))
            current_conflicts = count_conflicts(G, current_col)
        
        # aktualizuj ak pomoze znizit konflikty
        if current_conflicts < best_conflicts:
            best_conflicts = current_conflicts
            best_col = current_col.copy()
        
        conflict_history.append(current_conflicts)
    
    # nenaslo sa riesenie
    return best_col, best_conflicts == 0

def tabu_search(G, col, k, max_steps, tabu_tenure=10):
    # inspiracia: https://www.researchgate.net/publication/37437164_Werra_D_Using_Tabu_Search_Techniques_for_Graph_Coloring_Computing_39_345-351
    # tabu hladanie (trigger warning)
    # tato funkcia si nechava list, tzv. tabu lebo niekedy prijme 'horsie' riesenie, co "nechceme"
    # to znamena: ze ak v tomto liste uz nieco je, tak je to 'tabu' lebo sme to uz skusili, tak sa to preskoci
    n = len(G.nodes())
    current_col = col.copy()
    
    best_col = col.copy()
    best_conflicts = count_conflicts(G, col)
    current_conflicts = best_conflicts
    
    # tabu list: (node, color) -> step when it's no longer tabu
    tabu_list = {}
    
    for step in range(max_steps):
        # Show progress
        if step % 500 == 0:
            print(f"Tabu Search Step {step}: Best conflicts = {best_conflicts}")
            
        # If no conflicts, we found a valid coloring
        if best_conflicts == 0:
            return best_col, True
            
        # Find the best non-tabu move or a tabu move that improves the best solution
        best_move = None
        best_move_conflicts = float('inf')
        
        # Focus on conflicting nodes
        conflicting_nodes = get_conflicting_nodes(G, current_col)
        nodes_to_check = list(conflicting_nodes) if conflicting_nodes else list(G.nodes())
        
        # max 20 nodes nech nam nerastu sive fuzy
        if len(nodes_to_check) > 20:
            nodes_to_check = random.sample(nodes_to_check, 20)
        
        # eval moznych moznosti
        for node in nodes_to_check:
            current_color = current_col[node]
            
            for new_color in range(k):
                if new_color == current_color:
                    continue
                    
                # kontrolujeme ci sa to opakuje, ak nie rob krok
                is_tabu = tabu_list.get((node, new_color), -1) > step
                
                # docasne zmen farbu
                current_col[node] = new_color
                conflicts = count_conflicts(G, current_col)
                
                # prijat ak:
                # 1. neni tabu and lepsie ako doterajsi krok
                # 2. tabu ale lepsie ako riesenie doteraz (aspiracia)
                if (not is_tabu and conflicts < best_move_conflicts) or (conflicts < best_conflicts):
                    best_move = (node, new_color)
                    best_move_conflicts = conflicts
                
                # nasad nazad farbu ked neni dobre
                current_col[node] = current_color
        
        # ak sa nic nenaslo, diverzifikovat
        if best_move is None:
            if step % 500 == 0:
                print(f"No better move found at step {step}. Walking.")
            current_col = random_walk(G, current_col, k, steps=n//5)
            current_conflicts = count_conflicts(G, current_col)
        else:
            # uplatni najlepsi krok
            node, new_color = best_move
            old_color = current_col[node]
            current_col[node] = new_color
            current_conflicts = best_move_conflicts
            
            # pridaj spiatocny krok do tabu listu
            tabu_list[(node, old_color)] = step + tabu_tenure + random.randint(0, 5)  # pridat nahodnost
            
            # aktualizuj najlepsie riesenie ak sa najde
            if current_conflicts < best_conflicts:
                best_conflicts = current_conflicts
                best_col = current_col.copy()
    
    return best_col, best_conflicts == 0

def simulated_annealing(G, col, k, max_steps, initial_temp=1.0, cooling_rate=0.9995):
    # simulovanie "kalenie" (fun fact, stara mama dakedy robila na kaliarni)
    # tato funkcia dokaze odist z lokalneho minima vdaka delte teploty
    # inspiracia: https://github.com/engkareeem/Simulated-Annealing-Graph-Coloring/blob/main/src/logic/simulated_annealing.ts
    n = len(G.nodes())
    current_col = col.copy()
    current_conflicts = count_conflicts(G, current_col)
    
    best_col = current_col.copy()
    best_conflicts = current_conflicts
    temperature = initial_temp
    
    no_improvement = 0
    max_no_improvement = 500
    
    # nechame najlepsie riesenia aby sme sa vratili ked klesne teplota
    best_solutions = [(current_conflicts, current_col.copy())]
    
    for step in range(max_steps):
        # progress tracking
        if step % 500 == 0:
            print(f"Annealing Step {step}: Conflicts = {current_conflicts}, Best = {best_conflicts}, Temp = {temperature:.4f}")
            
        # ak najdeme riesenie
        if current_conflicts == 0:
            return current_col, True
            
        # znovunahrievanie ak je to stuck
        if no_improvement > max_no_improvement:
            print(f"Reheating at step {step} - no improvement for {no_improvement} steps")
            temperature = initial_temp * 0.8  # nie na initial temp
            
            # logika najlepsieho riesenia
            if len(best_solutions) > 1:
                # s tymto pomahal gpt, je to nejaky vyber ktory prioritizuje lepsi, ale stale nahodny
                solution_idx = min(int(random.expovariate(1.0) * len(best_solutions)), len(best_solutions) - 1)
                _, saved_col = best_solutions[solution_idx]
                current_col = saved_col.copy()
                current_conflicts = count_conflicts(G, current_col)
                print(f"Returned to saved solution with {current_conflicts} conflicts")
            
            no_improvement = 0
        else:
            temperature *= cooling_rate
    
        if temperature < 0.01:
            temperature = 0.01  # minimum temp
        
        # hlavne nad conflicting nodes
        conflicting_nodes = list(get_conflicting_nodes(G, current_col))
        
        if conflicting_nodes and current_conflicts < 5:
            node = random.choice(conflicting_nodes)
            current_node_conflicts = sum(1 for nbr in G.neighbors(node) if current_col[nbr] == current_col[node])
            
            best_color = current_col[node]
            best_node_conflicts = current_node_conflicts
            
            for color in range(k):
                if color == current_col[node]:
                    continue
                
                current_col[node] = color
                new_node_conflicts = sum(1 for nbr in G.neighbors(node) if current_col[nbr] == color)
                
                if new_node_conflicts < best_node_conflicts:
                    best_node_conflicts = new_node_conflicts
                    best_color = color
            
            if best_node_conflicts < current_node_conflicts or random.random() < math.exp(-(best_node_conflicts - current_node_conflicts) / temperature):
                current_col[node] = best_color
                current_conflicts = count_conflicts(G, current_col)
            else:
                current_col[node] = best_color if best_node_conflicts < current_node_conflicts else current_col[node]
        else:
            if conflicting_nodes:
                node = random.choice(conflicting_nodes)
            else:
                node = random.choice(list(G.nodes()))
            
            current_color = current_col[node]
            
            neighbor_colors = {current_col[nbr] for nbr in G.neighbors(node)}
            available_colors = list(set(range(k)) - neighbor_colors)
            
            if available_colors and random.random() < 0.9:  # ak je mozne vyber nekonfliktujucu farbu
                new_color = random.choice(available_colors)
            else:
                colors = [c for c in range(k) if c != current_color]
                new_color = random.choice(colors) if colors else current_color
            
            # konflikty
            old_conflicts = sum(1 for nbr in G.neighbors(node) if current_col[nbr] == current_color)
            
            # farba
            current_col[node] = new_color
            new_conflicts = sum(1 for nbr in G.neighbors(node) if current_col[nbr] == new_color)
            
            # delta
            delta = new_conflicts - old_conflicts
            
            # vypocet sance prijatie riesenia
            if delta <= 0 or random.random() < math.exp(-delta / max(0.01, temperature)):
                # prijat a prepocitat konflikty
                current_conflicts = count_conflicts(G, current_col)
            else:
                # neprijat
                current_col[node] = current_color
        
        # aktualizuj riesenie ak jedobre
        if current_conflicts < best_conflicts:
            best_conflicts = current_conflicts
            best_col = current_col.copy()
            no_improvement = 0
            
            # pridat do rieseni
            best_solutions.append((current_conflicts, current_col.copy()))
            # nechavame najlepsie riesenia - tuple sorting, lambda = 1st element (toto tiez robil chatCBT)
            best_solutions = sorted(best_solutions, key=lambda x: x[0])[:5]
            
            print(f"New best at step {step}: {best_conflicts} conflicts")
        else:
            no_improvement += 1
    
    return best_col, best_conflicts == 0


def iscoloring(G, col):
    # podla zadania ulohy - kontrola ci je farbenie grafu spravne (nie su 2 rovnake farby pri sebe)
    for u, v in G.edges():
        if col[u] == col[v]:
            return False
    return True

def color(G, k, steps):
    # tato funkcia farbi graf / snazi sa nafarbit graf s K farbami
    n = len(G.nodes())
    print(f"Attempting to color graph with {k} colors (nodes: {n}, edges: {G.number_of_edges()})")
    
    # zaciname s greedy coloring aby to bolo rychlejsie / lepsia starting position
    col = [0] * n  # nastav vsetky nodes s farbou 0
    
    # greedy approach
    for node in range(n):
        # zober farby susedov
        neighbor_colors = {col[nbr] for nbr in G.neighbors(node) if nbr < node}
        
        # najdi prvu moznu farbu
        for color in range(k):
            if color not in neighbor_colors:
                col[node] = color
                break
    
    # vypocet prvotnych konfliktov
    initial_conflicts = count_conflicts(G, col)
    print(f"Initial greedy coloring has {initial_conflicts} conflicts")
    
    # distribuujeme kroky medzi algoritmy
    hill_climbing_steps = int(steps * 0.4)  # 40% pre hill climb
    tabu_steps = int(steps * 0.4)           # 40% pre tabu search
    sa_steps = steps - hill_climbing_steps - tabu_steps  # zvysne pre annealing
    
    # fhase 1: hill climb s random walks
    print(f"Phase 1: Hill Climbing with Random Walks ({hill_climbing_steps} steps)")
    col, success = hill_climbing_with_walks(G, col, k, hill_climbing_steps)
    
    if success:
        print(f"Successfully found coloring with {k} colors using hill climbing!")
        return col, True
    
    # faza 2: tabu search
    print(f"Phase 2: Tabu Search ({tabu_steps} steps)")
    col, success = tabu_search(G, col, k, tabu_steps)
    
    if success:
        print(f"Successfully found coloring with {k} colors using tabu search!")
        return col, True
    
    # faza 3: simulated annealing
    print(f"Phase 3: Simulated Annealing ({sa_steps} steps)")
    col, success = simulated_annealing(G, col, k, sa_steps, initial_temp=1.0, cooling_rate=0.9995)
    
    if success:
        print(f"Successfully found coloring with {k} colors using simulated annealing!")
        return col, True
    
    # ak unsuccessful :/ tak vratime posledne validne farbenie
    conflicts = count_conflicts(G, col)
    print(f"Failed to find valid coloring. Best solution has {conflicts} conflicts.")
    return col, False

# -------------------------------------------------------------------------------------
#
#                           ##     ##    ###    #### ##    ## 
#                           ###   ###   ## ##    ##  ###   ## 
#                           #### ####  ##   ##   ##  ####  ## 
#                           ## ### ## ##     ##  ##  ## ## ## 
#                           ##     ## #########  ##  ##  #### 
#                           ##     ## ##     ##  ##  ##   ### 
#                           ##     ## ##     ## #### ##    ## 
#
# -------------------------------------------------------------------------------------

if __name__ == "__main__":
    # nacitaj graf z internetu
    url = "https://cedric.cnam.fr/~porumbed/graphs/dsjc125.9.col"
    print(f"Using graph from {url}")
    G = load_web_graph(url)
    
    # vytvor zaindexovany graf
    G_relabeled = nx.convert_node_labels_to_integers(G)
    print(f"Graph loaded with {G_relabeled.number_of_nodes()} nodes and {G_relabeled.number_of_edges()} edges")
    
    # najdi ofarbenie a postupne znizuj 'k' t.j. pocet farieb
    k_values = [46, 45, 44, 43]  # zacni s vyssim 'k' a znizuj
    max_steps = 30000  # menit podla potreby
    
    for k in k_values:
        print(f"\n===== Attempting coloring with k={k} colors =====")
        start_time = time.time()
        
        col, success = color(G_relabeled, k, max_steps) # hlavna hviezda
        
        elapsed = time.time() - start_time
        
        if success:
            print(f"[SUCCESS]: Found valid coloring with {k} colors in {elapsed:.2f} seconds!")
            print("[COLORING]: ", col)
            # kontrola
            verification = iscoloring(G_relabeled, col) 
            print(f"Color correctness: {verification}")
        else:
            print(f"[FAILURE]: Could not find valid coloring with {k} colors after {elapsed:.2f} seconds.")
            print("Stopping search. Minimum colors found: ", k+1 if k < k_values[0] else "none")
            break

Using graph from https://cedric.cnam.fr/~porumbed/graphs/dsjc125.9.col
Graph loaded with 125 nodes and 6961 edges

===== Attempting coloring with k=46 colors =====
Attempting to color graph with 46 colors (nodes: 125, edges: 6961)
Initial greedy coloring has 107 conflicts
Phase 1: Hill Climbing with Random Walks (12000 steps)
Step 0: Best conflicts = 107, Current conflicts = 107
Step 500: Best conflicts = 14, Current conflicts = 18
Step 1000: Best conflicts = 10, Current conflicts = 64
Step 1500: Best conflicts = 10, Current conflicts = 16
Step 2000: Best conflicts = 6, Current conflicts = 56
Step 2500: Best conflicts = 6, Current conflicts = 16
Step 3000: Best conflicts = 5, Current conflicts = 75
Step 3500: Best conflicts = 5, Current conflicts = 20
Step 4000: Best conflicts = 5, Current conflicts = 65
Step 4500: Best conflicts = 4, Current conflicts = 17
Step 5000: Best conflicts = 4, Current conflicts = 60
Step 5500: Best conflicts = 4, Current conflicts = 23
Step 6000: Best confli